In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from technicals.indicators import macd
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0


def apply_signal(df):
    df['signal'] = np.where((df['direction'] == BUY) & (df['mid_l'] > df['ema_100']) & (df['candle_dir'] == BUY), BUY,
                            np.where((df['direction'] == SELL) & (df['mid_h'] < df['ema_100']) & (df['candle_dir'] == SELL), SELL, NONE))
    return df['signal']
  

In [4]:
pair = 'USD_CHF'

df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")

df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = macd(df_an)
df_an['ema_100'] = df_an['mid_c'].ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)


In [6]:
df_an['MACD_delta'] = df_an['MACD'] - df_an['signal']
df_an['MACD_delta_prev'] = df_an['MACD_delta'].shift(1)
df_an['direction'] = np.where((df_an['MACD_delta'] > 0) & (df_an['MACD_delta_prev'] < 0), BUY,
                         np.where((df_an['MACD_delta'] < 0) & (df_an['MACD_delta_prev'] > 0), SELL, NONE))

candle_dir = df_an['mid_c'] - df_an['mid_o']
df_an['candle_dir'] = [BUY if x >= 0 else SELL for x in candle_dir]


In [7]:
df_an['direction'].value_counts()

direction
 0    48040
 1     2110
-1     2110
Name: count, dtype: int64

In [8]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_l,ask_c,MACD,signal,histogram,ema_100,MACD_delta,MACD_delta_prev,direction,candle_dir
0,2016-01-08 01:00:00+00:00,1958,0.99402,0.99844,0.99380,0.99684,0.99386,0.99824,0.99360,0.99668,...,0.99394,0.99700,-0.002987,-0.002745,-0.000243,1.003354,-0.000243,NaN,0,1
1,2016-01-08 02:00:00+00:00,782,0.99677,0.99737,0.99610,0.99652,0.99659,0.99719,0.99593,0.99638,...,0.99625,0.99667,-0.002773,-0.002750,-0.000022,1.003198,-0.000022,-0.000243,0,-1
2,2016-01-08 03:00:00+00:00,280,0.99660,0.99719,0.99624,0.99633,0.99643,0.99703,0.99610,0.99619,...,0.99638,0.99647,-0.002588,-0.002718,0.000130,1.003042,0.000130,-0.000022,1,-1
3,2016-01-08 04:00:00+00:00,207,0.99632,0.99768,0.99632,0.99744,0.99618,0.99750,0.99618,0.99728,...,0.99647,0.99759,-0.002325,-0.002639,0.000314,1.002915,0.000314,0.000130,0,1
4,2016-01-08 05:00:00+00:00,320,0.99744,0.99747,0.99683,0.99726,0.99730,0.99733,0.99669,0.99713,...,0.99697,0.99740,-0.002107,-0.002533,0.000426,1.002787,0.000426,0.000314,0,-1


In [11]:
def run_pair(pair, time_delta=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_delta}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    
    df_an = macd(df_an)
    df_an['ema_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()

    df_an['MACD_delta'] = df_an['MACD'] - df_an['signal']
    df_an['MACD_delta_prev'] = df_an['MACD_delta'].shift(1)
    df_an['direction'] = np.where((df_an['MACD_delta'] > 0) & (df_an['MACD_delta_prev'] < 0), BUY,
                             np.where((df_an['MACD_delta'] < 0) & (df_an['MACD_delta_prev'] > 0), SELL, NONE))
    candle_dir = df_an['mid_c'] - df_an['mid_o']
    df_an['candle_dir'] = [1 if x >= 0 else -1 for x in candle_dir]
    
    
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 
                'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 
                'ask_l', 'ask_c', 'direction', 'candle_dir', 'ema_100']
    
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    df_slim.reset_index(drop=True, inplace=True)
    
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_delta=time_delta
    )
    
    gt.run_test()
    return gt.df_results
    

In [12]:
results = []
for pair in ['USD_CHF', 'USD_JPY', 'GBP_USD']:
    results.append(dict(pair=pair, result=run_pair(pair, time_delta=1)))

In [13]:
for result in results:
    print(result['pair'], result['result'].result.sum())

USD_CHF -486.5
USD_JPY -252.5
GBP_USD -289.0


In [ ]:
results = []
for pair in ['USD_CHF', 'USD_JPY', 'GBP_USD']:
    results.append({
        'pair': pair,
        'result': run_pair(pair, time_delta=4)
    })

for result in results:
    print(result['pair'], result['result'].result.sum())
    